In [278]:
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression, Ridge, Lasso, RidgeCV, LassoCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PolynomialFeatures
import pandas as pd
import numpy as np

In [131]:
RANDOM_STATE = 42

In [132]:
dataset = load_boston()
X = pd.DataFrame(dataset.data)
X.columns = dataset.feature_names
y = dataset.target

1. Разделите выборку на обучающую и тестовую в отношении 80%/20%

In [133]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = RANDOM_STATE)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((404, 13), (102, 13), (404,), (102,))

In [134]:
X.shape

(506, 13)

2. Обучите стандартную регрессию, а также Ridge и  Lasso и параметрами по умолчанию и выведите их R2 на тестовой выборке

In [37]:
# Study linear regression
%time
lr = LinearRegression()
lr.fit(X_train, y_train)

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.01 µs


LinearRegression()

In [38]:
y_pred = lr.predict(X_test)
r2_lin = r2_score(y_test, y_pred)

In [30]:
# Study ridge regression
%time
ridge = Ridge()
ridge.fit(X_train, y_train)

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 4.77 µs


Ridge()

In [39]:
y_pred = ridge.predict(X_test)
r2_r = r2_score(y_test, y_pred)

In [40]:
# Study lasso model
%time
lasso = Lasso()
lasso.fit(X_train, y_train)

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 5.72 µs


Lasso()

In [42]:
y_pred = lasso.predict(X_test)
r2_l = r2_score(y_test, y_pred)

In [47]:
print(f'Linear R2: {r2_lin:.4f}')
print(f'Ridge R2: {r2_r:.4f}')
print(f'Lasso R2: {r2_l:.4f}')

Linear R2: 0.6688
Ridge R2: 0.6662
Lasso R2: 0.6671


3. Для Ridge и Lasso подберите коэффициент регуляризации(используйте GridSearchCV, RidgeCV, LassoCV) в пределах от $10^{-5}$ до $10^5$ (по степеням 10). Посчитайте R2 на тестовой выборке по лучшим моделям и сравните с предыдущими результатами. Напишите как изменился результат

In [224]:
# Зададим параметры 
params = {'alpha': [10**level for level in range(-5,6)]}
print(params)

{'alpha': [1e-05, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000, 100000]}


**Ridge GridSearchCV**

In [64]:
ridge = Ridge()

ridge_gs = GridSearchCV(ridge, params)
ridge_gs.fit(X_train, y_train)
ridge_gs.best_params_

{'alpha': 1e-05}

In [65]:
ridge_gs = ridge_gs.best_estimator_
ridge_gs

Ridge(alpha=1e-05)

In [66]:
y_pred = ridge_gs.predict(X_test)
r2_rgs = r2_score(y_test, y_pred)
print(r2_rgs)

0.6687594856409732


**Ridge RidgeCV**

In [121]:
ridge_rcv = RidgeCV(alphas = params['alpha'])
ridge_rcv.fit(X_train, y_train)
ridge_rcv.alpha_

0.01

In [80]:
ridge_rcv = Ridge(alpha = ridge_rcv.alpha_)

In [84]:
ridge_rcv.fit(X_train, y_train)
y_pred = ridge_rcv.predict(X_test)

In [85]:
r2_rcv = r2_score(y_test, y_pred)
print(r2_rcv)

0.668750946206315


In [105]:
ridge_df = pd.DataFrame(data = [r2_r, r2_rgs, r2_rcv],
                        index = ['Ridge', 'Ridge GS', 'Ridge CV'], columns = ['Value'])
ridge_df['Diff_percent'] = round(ridge_df['Value']/ridge_df.loc['Ridge']['Value']*100 - 100, 4)
ridge_df

,Value,Diff_percent
Ridge,0.666222,0.0000
Ridge GS,0.668759,0.3809
Ridge CV,0.668751,0.3796


**Lasso GridSearchCV**

In [135]:
lasso = Lasso()

lasso_gs = GridSearchCV(lasso, params)
lasso_gs.fit(X_train, y_train)
lasso_gs.best_params_

{'alpha': 1e-05}

In [136]:
lasso_gs = lasso_gs.best_estimator_

In [137]:
y_pred = lasso_gs.predict(X_test)
r2_lgs = r2_score(y_test, y_pred)
print(r2_lgs)

0.6687598638315154


**Lasso LassoCV**

In [210]:
lasso_cv = LassoCV(alphas = params['alpha'])
lasso_cv.fit(X_train, y_train)

lasso_cv.alpha_

1e-05

In [211]:
lasso_cv = Lasso(alpha = lasso_cv.alpha_)
lasso_cv.fit(X_train, y_train)

Lasso(alpha=1e-05)

In [212]:
y_pred = lasso_cv.predict(X_test)
r2_lcv = r2_score(y_test, y_pred)
print(r2_lcv)

0.6687598638315154


In [292]:
lasso_cv = LassoCV(alphas = params['alpha'])
#lasso_cv.fit(X_train, y_train)

#lasso_cv.alpha_

KeyError: 'alpha'

In [293]:
lasso_cv.score(X_test, y_test)

0.6687598638315154

In [294]:
lasso_cv.score(X_train, y_train)

0.7508856354745458

In [168]:
y_pred = lasso_cv.predict(X_train)
r2_lcv = r2_score(y_train, y_pred)
print(r2_lcv)

0.7508856354745458


4. Проведите масштабирование выборки(используйте Pipeline, StandardScaler, MinMaxScaler), посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [184]:
# LinearRegression with StandardScaler()

pipe = Pipeline([('scaler', StandardScaler()), ('reg_model', LinearRegression())])
pipe.fit(X_train, y_train)
r2_lin_stscale = pipe.score(X_test, y_test)
r2_lin_stscale

0.668759493535632

In [185]:
# LinearRegression with MinMaxScaler()

pipe = Pipeline([('scaler', MinMaxScaler()), ('reg_model', LinearRegression())])
pipe.fit(X_train, y_train)
r2_lin_minmaxscale = pipe.score(X_test, y_test)
r2_lin_minmaxscale

0.668759493535632

In [195]:
scale_lin_df = pd.DataFrame(data = [r2_lin, r2_lin_stscale, r2_lin_minmaxscale],
                           index = ['wo/ scale', 'StandardScaler', 'MinMaxScaler'],
                           columns = ['Value'])
scale_lin_df['Diff_percent'] = round(scale_lin_df['Value']/scale_lin_df.loc['wo/ scale']['Value']*100 - 100, 4)
scale_lin_df


,Value,Diff_percent
wo/ scale,0.668759,0.0
StandardScaler,0.668759,-0.0
MinMaxScaler,0.668759,-0.0


Для линейной модели масштабирование ничего не изменило. Наоборот, знак минус показывает, что R2 ухудшилось незначительно

In [198]:
# Ridge with StandardScaler()

pipe = Pipeline([('scaler', StandardScaler()), ('reg_model', Ridge())])
pipe.fit(X_train, y_train)
r2_ridge_stscaler = pipe.score(X_test, y_test)
r2_ridge_stscaler

0.6684624359643561

In [199]:
# Ridge with MinMaxScaler()

pipe = Pipeline([('scaler', MinMaxScaler()), ('reg_model', Ridge())])
pipe.fit(X_train, y_train)
r2_ridge_minmaxscaler = pipe.score(X_test, y_test)
r2_ridge_minmaxscaler

0.6764100365423602

In [202]:
scale_ridge_df = pd.DataFrame(data = [r2_r, r2_ridge_stscaler, r2_ridge_minmaxscaler],
                             index = ['wo/ scale', 'StandardScaler', 'MinMaxScaler'],
                           columns = ['Value'])
scale_ridge_df['Diff_percent'] = round(scale_ridge_df['Value']/scale_ridge_df.loc['wo/ scale']['Value']*100 - 100, 4)
scale_ridge_df

,Value,Diff_percent
wo/ scale,0.666222,0.0000
StandardScaler,0.668462,0.3363
MinMaxScaler,0.676410,1.5292


Для Ridge модели масштабирование улучшает точность модели: лучше для этих данных подходит MinMaxScaler

In [203]:
# Lasso with StandardScaler()

pipe = Pipeline([('scaler', StandardScaler()), ('reg_model', Lasso())])
pipe.fit(X_train, y_train)
r2_lasso_stscaler = pipe.score(X_test, y_test)
r2_lasso_stscaler

0.6239428734251422

In [205]:
# Lasso with MinMaxScaler()

pipe = Pipeline([('scaler', MinMaxScaler()), ('reg_model', Lasso())])
pipe.fit(X_train, y_train)
r2_lasso_minmaxscaler = pipe.score(X_test, y_test)
r2_lasso_minmaxscaler

0.2573921442545194

In [207]:
scale_lasso_df = pd.DataFrame(data = [r2_l, r2_lasso_stscaler, r2_lasso_minmaxscaler],
                             index = ['wo/ scale', 'StandardScaler', 'MinMaxScaler'],
                             columns = ['Value'])
scale_lasso_df['Diff_percent'] = round(scale_lasso_df['Value']/scale_lasso_df.loc['wo/ scale']['Value']*100 - 100, 4)
scale_lasso_df

,Value,Diff_percent
wo/ scale,0.667145,0.0000
StandardScaler,0.623943,-6.4757
MinMaxScaler,0.257392,-61.4189


Для Lasso модели масштабирование серьезно ухудшает точность модели.

5. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [260]:
# Ridge model (StandardScaler)

params = {'reg_model__alpha': [10**x for x in range(-5,6)]}

pipe = Pipeline([('scaler', StandardScaler()), ('reg_model', Ridge())])

ridge_p_stscaler = GridSearchCV(pipe, params)
ridge_p_stscaler.fit(X_train, y_train)

r2_ridge_p_stscaler= ridge_p_stscaler.best_estimator_.score(X_test, y_test)
r2_ridge_p_stscaler

0.6684624359643561

In [261]:
# Ridge model (MinMaxScaler)

params = {'reg_model__alpha': [10**x for x in range(-5,6)]}

pipe = Pipeline([('scaler', MinMaxScaler()), ('reg_model', Ridge())])

ridge_p_minmaxscaler = GridSearchCV(pipe, params)
ridge_p_minmaxscaler.fit(X_train, y_train)

r2_ridge_p_minmaxscaler = ridge_p_minmaxscaler.best_estimator_.score(X_test, y_test)
r2_ridge_p_minmaxscaler

0.6700309977617653

In [273]:
scale_ridgegs_df = pd.DataFrame(data = [r2_rgs, r2_ridge_p_stscaler, r2_ridge_p_minmaxscaler],
                             index = ['GS wo/ scale', 'GS StandardScaler', 'GS MinMaxScaler'],
                           columns = ['Value'])
scale_ridgegs_df['Diff_percent'] = round(scale_ridgegs_df['Value']/scale_ridgegs_df.loc['GS wo/ scale']['Value']*100 - 100, 4)
scale_ridgegs_df

,Value,Diff_percent
GS wo/ scale,0.668759,0.0000
GS StandardScaler,0.668462,-0.0444
GS MinMaxScaler,0.670031,0.1901


In [274]:
# Lasso model (StandardScaler)

params = {'reg_model__alpha': [10**x for x in range(-5,6)]}

pipe = Pipeline([('scaler', StandardScaler()), ('reg_model', Lasso())])

lasso_p_stscaler = GridSearchCV(pipe, params)
lasso_p_stscaler.fit(X_train, y_train)

r2_lasso_p_stscaler= lasso_p_stscaler.best_estimator_.score(X_test, y_test)
r2_lasso_p_stscaler

0.668759038334717

In [275]:
# Lasso model (MinMaxScaler)

params = {'reg_model__alpha': [10**x for x in range(-5,6)]}

pipe = Pipeline([('scaler', MinMaxScaler()), ('reg_model', Lasso())])

lasso_p_minmaxscaler = GridSearchCV(pipe, params)
lasso_p_minmaxscaler.fit(X_train, y_train)

r2_lasso_p_minmaxscaler= lasso_p_minmaxscaler.best_estimator_.score(X_test, y_test)
r2_lasso_p_minmaxscaler

0.6687605073677361

In [276]:
scale_lassogs_df = pd.DataFrame(data = [r2_lgs, r2_lasso_p_stscaler, r2_lasso_p_minmaxscaler],
                             index = ['GS wo/ scale', 'GS StandardScaler', 'GS MinMaxScaler'],
                           columns = ['Value'])
scale_lassogs_df['Diff_percent'] = round(scale_lassogs_df['Value']/scale_lassogs_df.loc['GS wo/ scale']['Value']*100 - 100, 4)
scale_lassogs_df

,Value,Diff_percent
GS wo/ scale,0.668760,0.0000
GS StandardScaler,0.668759,-0.0001
GS MinMaxScaler,0.668761,0.0001


Для Ridge и Lasso использование масштабирования не дает значимого эффекта при поиске коэффицента регуляризации. 

6. Добавьте попарные произведения признаков и их квадраты (используйте PolynomialFeatures) на масштабированных признаках, посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [285]:
# Ridge with PolynomialFeatures and Scaling
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('preprocessor', PolynomialFeatures()),
    ('reg_model', Ridge())
])
pipe.fit(X_train, y_train)

r2_ridge_poly_sts = pipe.score(X_test, y_test)
r2_ridge_poly_sts


0.8162948990230013

In [287]:
# Ridge with PolynomialFeatures and Scaling
pipe = Pipeline([
    ('scaler', MinMaxScaler()),
    ('preprocessor', PolynomialFeatures()),
    ('reg_model', Ridge())
])
pipe.fit(X_train, y_train)

r2_ridge_poly_mms = pipe.score(X_test, y_test)
r2_ridge_poly_mms


0.8299337208033138

Для Ridge масштабирование вместе с PolynomialFeatures позволяет добиться наивысшей точности в R2 ~ 0.83

In [290]:
# Lasso with PolynomialFeatures and Scaling
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('preprocessor', PolynomialFeatures()),
    ('reg_model', Lasso())
])
pipe.fit(X_train, y_train)

r2_lasso_poly_sts = pipe.score(X_test, y_test)
r2_lasso_poly_sts


0.7322762586137164

In [291]:
# Lasso with PolynomialFeatures and Scaling
pipe = Pipeline([
    ('scaler', MinMaxScaler()),
    ('preprocessor', PolynomialFeatures()),
    ('reg_model', Lasso())
])
pipe.fit(X_train, y_train)

r2_lasso_poly_mms = pipe.score(X_test, y_test)
r2_lasso_poly_mms


0.2611262741735658

Для Lasso масштабирование вместе с PolynomialFeatures позволяет добиться наивысшей точности в R2 ~ 0.73 только при масшатибировании StandardScaler

7. Подберите наилучшую модель (используйте Pipeline, GridSearchSCV) подбирая тип регуляризации (L1,L2), коэффициент регуляризации, метод масштабирования и степень полинома в PolynomialFeatures. Выведите итоговые параметры и результат R2. Напишите как изменился R2 по сравнению с предыдущими экспериментами

http://archive.ics.uci.edu/ml/datasets/Adult

In [288]:
link = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/adult-all.csv'
data = pd.read_csv(link, header=None)

In [289]:
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


8. Разделите выборку на признаки и целевую переменную(колонка со зачениями {<=50K,>50K}). Замените целевую переменную на числовые значения.

9. Выясните, присутствуют ли в данных пропуски. Заполните их самыми частыми значениями (испольуйте SimpleImputer)

10. Выберите колонки с числовыми и категориальными переменными.

11. Создайте пайплайн по обработке колонок(используйте OneHotEncoder,MinMaxScaler).

12. Посчитайте метрики accuracy и f1_score на предсказании только самого частого класса в целевой переменной.

13. Посчитайте cross_val_score по алгоритмам LogisticRegression, SVC, LinearSVC по метрикам accuracy и f1_score.
Напишите удалось ли превзойти предыдущий результат.

14. Можно заметить что в данных присутствуют значения '?', замените их самыми частыми значениями (испольуйте SimpleImputer)

15. Посчитайте cross_val_score на новых данных. Напишите удалось ли улучшить результат.

16. Посчитайте cross_val_score, если просто удалить значения '?'. Напишите как изменился результат

 17. Посчитайте cross_val_score для RandomForestClassifier,GradientBoostingClassifier. Напишите как изменился результат и какой вывод можно из этого сделать.

18. Подберите наилучшую модель, подбирая методы обработки колонок - масштабирование признаков, кодирование признаков и заполнение пропусков. Параметры алгоритмов оставьте по умолчанию. Выведите итоговые параметры и результат accuracy и f1_score.